In [203]:
# for viginere
def numrep(string):
    string = string.upper()
    output = ""
    for s in string:
        n = ord(s) - 65
        if n >= 10:
            output += str(n)
        else :
            output += '0' + str(n)
    return output

def strrep(num):
    output = ""
    for n in range(len(num)-1):
        if n % 2 == 0 :
            output+= str(chr(65 + int(num[n:n+2])))
    return output

## RS A

In [204]:
from random import randrange

In [205]:
def get_u_r(num):
    u = 0
    num -= 1

    while True:
        u += 1
        num //= 2
        if u != 0 and num % 2 != 0:
            break

    return (u, num)

def modular_pow(base, exponent, modulus): #square and multiply
    if modulus == 1:
        return 0
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2 == 1):
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def inverse(ra, rb):
    if rb > ra:
        ra, rb = rb, ra

    modulos = ra
    mult = [(1, 0), (0, 1)]
    i = 2
    while True:
        # print(str(ra) + " = " + str(rb) + "*", end='')
        mod = ra % rb
        q = (ra - mod) // rb
        # print(str(q)+" + " + str(mod))
        ra = rb
        rb = mod
        mult = [
            (mult[1][0], mult[1][1]),
            ((-q * mult[1][0]) + mult[0][0], (-q * mult[1][1]) + mult[0][1])
        ]
        if mod == 0:
            # print("GCD = " + str(ra))
            if ra == 1:
                return mult[0][1] % modulos
            else:
                return -1
            
            
def miller_rabin(p, s):
    if p == 2:
        return True
    if p % 2 == 0:
        return False

    u, r = get_u_r(p)
    for i in range(s):
        a = randrange(2, p - 1)
        z = modular_pow(a, r, p)

        if z != 1 and z != (p - 1):
            for j in range(u):
                z = modular_pow(z, 2, p)
                if z == p - 1:
                    break
            else:
                return False

    return True            
            
def get_rand_prime(nbits=16):
    while True:
        p = randrange(2 ** nbits, 2 * 2 ** nbits)
        # print(p)
        if miller_rabin(p, 100):
            return p
def CRT(y, d, p, q):
    n = p * q

    # 1- Convert to CRT domain
    yp = y % p
    yq = y % q
    # print("(yp, yq) = ", str((yp, yq)))

    # 2- Do the computations
    dp = d % (p - 1)
    dq = d % (q - 1)
    # print("(dp, dq) = ", str((dp, dq)))

    xp = pow(yp, dp, p)
    xq = pow(yq, dq, q)
    # print("(xp, xq) = ", str((xp, xq)))

    # 3- Inverse transform
    inv = inverse(p, q)
    print(inv)
    cp = pow(q, p - 2, p)
    cq = pow(p, q - 2, q)
    # print(cq == pow(p, q-2, q))
    # print("(cp, cq) = ", str((p, q)))

    x = ((q * cp * xp) + (p * cq * xq)) % n
    # print("x = ", x, "mod " + str(n))
    return x

In [206]:
def msg_to_int(msg):
    int_msg = ""
    for ch in msg:
        pre = "{0:b}".format(ord(ch))
        if len(pre) < 7:
            pre = "0" * (7 - len(pre)) + pre
        int_msg += pre

    return int(int_msg, 2)

In [207]:
msg_to_int("ali")

1603177

In [208]:
def int_to_msg(i):
    bin_format = "{0:7b}".format(i)
    msg = ""

    for b in range(0, len(bin_format), 7):
        msg += chr(int(bin_format[b:b + 7], 2))

    return msg

In [209]:
int_to_msg(1074761)

'ALI'

In [210]:
def encryption(msg, e, n):
    int_msg = msg_to_int(msg)
    encrypted = modular_pow(int_msg, e, n)
    return encrypted

def decryption(msg, d, p, q):
    decrypted = CRT(int(msg), d, p, q)
    return int_to_msg(decrypted)

In [211]:
def RSA_Init(nbits=512):  # setup
    p = get_rand_prime(nbits)
    q = get_rand_prime(nbits)
    while p == q:
        q = get_rand_prime(nbits)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = randrange(2 ** 16, 2 ** 17)
    d = inverse(phi, e)
    while d == -1:
        e = randrange(2 ** 16, 2 ** 17)
        d = inverse(phi, e)

    return {
        "p": p,
        "q": q,
        "n": n,
        "phi": phi,
        "e": e,
        "d": d
    }

In [212]:
RSAparams = RSA_Init(512)

In [213]:
RSAparams

{'p': 25389859574812623266407888161036107033628787976217847935425291027026324923299909576388858622986513106539782080393126395399686580482730389686712289048744751,
 'q': 23859206077360364816838553567888649481402602749257891222102232879156192845192047738920126379845973525255791906001915563280718706432435605951514908842410099,
 'n': 605781891870695589269424621725056351291994448025258500647277435857546215245083969831684600854816482463030756602071144275472009700037864145058070372783268690974233728169450910944902916070874970566984301993464504178238839387180392105400449758599989216677318732996779458603518765725584792237002665317589515640349,
 'phi': 605781891870695589269424621725056351291994448025258500647277435857546215245083969831684600854816482463030756602071144275472009700037864145058070372783268641725168075996462827698461187146118455535593576517725346650714933204662623613443134449614986384190686937422793063561560085320297877071007027090391624485500,
 'e': 105263,
 'd': 250167094227023144

In [214]:
message = "hello world yoo"

In [220]:
cipher = encryption(
    message, 
    RSAparams['e'], 
    RSAparams['n']
)

In [221]:
cipher

41538284339137666324408118553659370386853467487697817810583930601974806453019045560171811406497052182544578247662710336800447825066780958083232072572991467188387473631824797778307856827855492190549477829553368272749906294240550681870801544142693452285119033301229599459783598843947922169068130765735993115828

In [222]:
plain = decryption(
    encrypt,
    RSAparams['d'], 
    RSAparams['p'], 
    RSAparams['q']
)

14700837478241349207771742350628766531301249374499565565433751081414710245083992637571926584608511125548317702137149514148129160029552286340969193439087896


In [223]:
cipher

41538284339137666324408118553659370386853467487697817810583930601974806453019045560171811406497052182544578247662710336800447825066780958083232072572991467188387473631824797778307856827855492190549477829553368272749906294240550681870801544142693452285119033301229599459783598843947922169068130765735993115828